<a href="https://colab.research.google.com/github/usamabaig1/Portfolio_Optimization/blob/main/PortfolioOptimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cvxopt as opt
from cvxopt import blas, solvers
import pandas as pd

np.random.seed(123)

# Turn off progress printing 
solvers.options['show_progress'] = False

In [4]:
!pip install pyportfolioopt
import pypfopt 
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

     |████████████████████████████████| 51kB 2.8MB/s 


In [62]:
data  = pd.read_excel('Data.xlsx', sheet_name= 'Portfolios')
covMat  = pd.read_excel('Data.xlsx', sheet_name= 'Covariance Matrix', index_col=0)
portfolios = 10
command = 'all'
# maxMean, minVolatility, maxSharpRatio, all

# print(df.head())
print(data.shape)

# print(covM.head())
print(covM.shape)

(199, 3)
(199, 199)


In [63]:
def maxMean(dfView, X):
  df = dfView.copy()
  df.sort_values(by='Mean', inplace=True, ascending=False)
  df.reset_index(inplace=True, drop=True)
  df['Weight'] = 0
  ports = []
  for x in range(len(df)):
    if x < X:
      df.loc[x,'Weight'] = 1
      ports += [df.loc[x,'Port']]
    else:
      break
  print("maxMean portfolios:", sorted(ports))     
  df.sort_values(by='Port', inplace=True, ascending=True)
  df.reset_index(inplace=True, drop=True)
  # df.to_csv("maxMean.csv")
  return df


In [64]:
def minVolatility(dfView, covM, X):
  df = dfView.copy()
  ef = EfficientFrontier(df.Mean, covM)
  weights = ef.min_volatility()
  cleaned_weights = ef.clean_weights()
  df['volatility_Weight'] = 0

  for x in range(len(cleaned_weights)):
    df.loc[x,'volatility_Weight'] = cleaned_weights[x]

  df.sort_values(by='volatility_Weight', inplace=True, ascending=False)
  df.reset_index(inplace=True, drop=True)

  df['Weight'] = 0
  ports = []
  for x in range(len(df)):
    if x < X:
      df.loc[x,'Weight'] = 1
      ports += [df.loc[x,'Port']]
    else:
      break
  print("minVolatility portfolios:", sorted(ports))     
  df.sort_values(by='Port', inplace=True, ascending=True)
  df.reset_index(inplace=True, drop=True)
  df.drop(columns='volatility_Weight', axis=1, inplace=True)
  # df.head(X+2)
  # df.to_csv("minVolatility.csv")
  return df



In [65]:
def maxSharpRatio(dfView, covM, X):
  # get weights maximizing the Sharpe ratio
  df = dfView.copy()
  ef = EfficientFrontier(df.Mean, covM)
  weights = ef.max_sharpe()
  cleaned_weights = ef.clean_weights()
  df['sharpe_Weight'] = 0

  for x in range(len(cleaned_weights)):
    df.loc[x,'sharpe_Weight'] = cleaned_weights[x]

  df.sort_values(by='sharpe_Weight', inplace=True, ascending=False)
  df.reset_index(inplace=True, drop=True)

  df['Weight'] = 0
  ports = []
  for x in range(len(df)):
    if x < X:
      df.loc[x,'Weight'] = 1
      ports += [df.loc[x,'Port']]
    else:
      break
  print("maxSharpRatio portfolios:", sorted(ports))     
  df.sort_values(by='Port', inplace=True, ascending=True)
  df.reset_index(inplace=True, drop=True)
  df.drop(columns='sharpe_Weight', axis=1, inplace=True)
  # df.head(X+2)
  # df.to_csv("maxSharpRatio.csv")
  return df



In [66]:
if command == 'maxMean':
  maxMean(data, portfolios).to_csv("maxMean.csv", index=False)
elif command == 'minVolatility':
  minVolatility(data, covMat, portfolios).to_csv("minVolatility.csv", index=False)
elif command == 'maxSharpRatio':
  maxSharpRatio(data, covMat, portfolios).to_csv("maxSharpRatio.csv", index=False)
elif command == 'all':
  dfM = maxMean(data, portfolios)
  dfV = minVolatility(data, covMat, portfolios)
  dfS = maxSharpRatio(data, covMat, portfolios)
  data['sharpe_Weight'] = dfS['Weight']
  data['volatility_Weight'] = dfV['Weight']
  data['maxMean_Weight'] = dfM['Weight']
  data.to_csv('All.csv', index=False)
else:
  print ('Invalid command.')

maxMean portfolios: [6, 11, 24, 33, 39, 46, 50, 61, 87, 186]
minVolatility portfolios: [25, 27, 55, 81, 120, 133, 146, 169, 172, 188]
maxSharpRatio portfolios: [22, 25, 27, 55, 81, 120, 133, 169, 172, 188]
